In [1]:
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import os
import scipy.io as sio
import scipy.misc
import fnmatch
import random
import math
import cv2
from sklearn.utils import shuffle

In [2]:
# Generate images?
generate = True
# Preprocess data?
split = True

call_folder = '/home/dario/Documents/SemThes_Local/Images_RAD/'


store_folder = ''

# FOR IMAGE GENERATION ----------------------
# Batch size of images
X_shape = 224
Y_shape = 224

# alpha = np.array([[0.501, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.599]])
alpha = np.array([[0.501]])
r_mean = 615
g_mean = 530
b_mean = 465
sigma = 28

# classes
classes = 10

# FOR PREPROCESSING ----------------------
# How much data to use?
use_data = 1.

# Divison factor for Training, Validation and Test data [0,1]:
Train_split = 8./10
Val_split = 1./10
Test_split = 1./10

In [3]:
_, nr_param = alpha.shape
nr_hyp_images = 1
batch_counter = 1

CSS_calc = np.full((3, 31), 0.)
CSS = np.full((int(np.floor(1392/X_shape)) * int(np.floor(1300/Y_shape)) * nr_hyp_images * nr_param, 2), 0.)

for i in range(0, nr_hyp_images):
    print('Evaluating Image ' + str(i+1))
    mat_contents = sio.loadmat(os.path.join(call_folder, 'RAD_' + str(i+1) + '.mat'))
    rad =mat_contents['rad_new']

    x_row, y_col, spect = rad.shape
    X_window = int(np.floor(x_row / X_shape))
    Y_window = int(np.floor(y_col / Y_shape))

    rad = rad[0:X_window * X_shape, 0:Y_window * Y_shape, :]
    x_row, y_col, spect = rad.shape
    n_features = x_row * y_col

    rad_reshaped = np.reshape(rad, (1, n_features, spect))[0]
    rad_reshaped = np.swapaxes(rad_reshaped, 0, 1)

    # Compute RGB image from Power Spectrum and CSS
    for counter in range(0, nr_param):
        q = 0
        for i in range(401, 711, 10):
            CSS_calc[0,q] = alpha[0, counter] * math.exp(-(i-r_mean) ** 2 / (2 * sigma ** 2))
            CSS_calc[1,q] = alpha[0, counter] * math.exp(-(i-g_mean) ** 2 / (2 * sigma ** 2))
            CSS_calc[2,q] = alpha[0, counter] * math.exp(-(i-b_mean) ** 2 / (2 * sigma ** 2))
            q = q + 1

        # I = [3 x m] || CSS_new = [3 x 33] || rad_reshaped = [33 x m]
        I = np.matmul(CSS_calc, rad_reshaped) / 4095

        I_image = np.reshape(I, (3, x_row, y_col))
        I_image = np.swapaxes(I_image, 0, 1)
        I_image = np.swapaxes(I_image, 1, 2)
        I_image[I_image > 1] = 1

        # Now store batches of Images
        scipy.misc.toimage(I_image, cmin=0, cmax=1).save(os.path.join(store_folder, str(batch_counter) + '.jpg'))

print('Done!')



Evaluating Image 1
Done!


/home/dario/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


In [4]:
print(I_image) # original

[[[0.245588   0.23060621 0.21368638]
  [0.38645873 0.33702269 0.30858411]
  [0.35191289 0.31523688 0.29923018]
  ...
  [0.66106272 0.59139061 0.51676534]
  [0.65989589 0.59359856 0.51884828]
  [0.66271737 0.59222512 0.51811314]]

 [[0.24558116 0.23744195 0.21648025]
  [0.30045435 0.28551381 0.31226106]
  [0.29513633 0.2797777  0.30417641]
  ...
  [0.67551459 0.60264015 0.52281201]
  [0.67806352 0.60379216 0.52365583]
  [0.68116851 0.60482636 0.52483891]]

 [[0.24658255 0.23872793 0.2174391 ]
  [0.29069708 0.29699377 0.33824093]
  [0.28132905 0.28435299 0.32578379]
  ...
  [0.70454658 0.62027316 0.53240914]
  [0.70423463 0.62281988 0.53354277]
  [0.70972093 0.62346379 0.53374661]]

 ...

 [[0.22693758 0.21751106 0.19954304]
  [0.23713898 0.23628966 0.19653312]
  [0.22950362 0.22858966 0.19243216]
  ...
  [0.29645469 0.29494684 0.22059554]
  [0.30039904 0.30037833 0.22193909]
  [0.29566225 0.29840586 0.22123062]]

 [[0.22599375 0.21383926 0.19691006]
  [0.23482367 0.23368523 0.19492184]


In [5]:
img = np.array(Image.open('1.jpg'), dtype=np.float16)
print(img)

[[[ 61.  65.  64.]
  [ 81.  85.  84.]
  [ 83.  85.  84.]
  ...
  [171. 149. 135.]
  [171. 149. 135.]
  [171. 149. 135.]]

 [[ 51.  55.  56.]
  [ 74.  78.  79.]
  [ 72.  74.  73.]
  ...
  [173. 151. 137.]
  [174. 152. 138.]
  [174. 152. 138.]]

 [[ 54.  58.  59.]
  [ 82.  86.  87.]
  [ 69.  70.  72.]
  ...
  [179. 158. 139.]
  [179. 158. 139.]
  [179. 158. 139.]]

 ...

 [[ 59.  59.  49.]
  [ 59.  59.  49.]
  [ 59.  59.  51.]
  ...
  [ 72.  75.  58.]
  [ 73.  76.  59.]
  [ 72.  75.  58.]]

 [[ 57.  57.  47.]
  [ 58.  58.  48.]
  [ 57.  57.  49.]
  ...
  [ 71.  74.  57.]
  [ 72.  75.  58.]
  [ 71.  74.  57.]]

 [[ 56.  56.  46.]
  [ 57.  57.  47.]
  [ 57.  57.  49.]
  ...
  [ 71.  74.  57.]
  [ 71.  74.  57.]
  [ 70.  73.  56.]]]


In [6]:
_, nr_param = alpha.shape
nr_hyp_images = 1
batch_counter = 1

CSS_calc = np.full((3, 31), 0.)
CSS = np.full((int(np.floor(1392/X_shape)) * int(np.floor(1300/Y_shape)) * nr_hyp_images * nr_param, 2), 0.)

for i in range(0, nr_hyp_images):
    print('Evaluating Image ' + str(i+1))
    mat_contents = sio.loadmat(os.path.join(call_folder, 'RAD_' + str(i+1) + '.mat'))
    rad =mat_contents['rad_new']

    x_row, y_col, spect = rad.shape
    X_window = int(np.floor(x_row / X_shape))
    Y_window = int(np.floor(y_col / Y_shape))

    rad = rad[0:X_window * X_shape, 0:Y_window * Y_shape, :]
    x_row, y_col, spect = rad.shape
    n_features = x_row * y_col

    rad_reshaped = np.reshape(rad, (1, n_features, spect))[0]
    rad_reshaped = np.swapaxes(rad_reshaped, 0, 1)

    # Compute RGB image from Power Spectrum and CSS
    for counter in range(0, nr_param):
        q = 0
        for i in range(401, 711, 10):
            CSS_calc[0,q] = alpha[0, counter] * math.exp(-(i-r_mean) ** 2 / (2 * sigma ** 2))
            CSS_calc[1,q] = alpha[0, counter] * math.exp(-(i-g_mean) ** 2 / (2 * sigma ** 2))
            CSS_calc[2,q] = alpha[0, counter] * math.exp(-(i-b_mean) ** 2 / (2 * sigma ** 2))
            q = q + 1

        # I = [3 x m] || CSS_new = [3 x 33] || rad_reshaped = [33 x m]
        I = np.matmul(CSS_calc, rad_reshaped) / 4095

        I_image = np.reshape(I, (3, x_row, y_col))
        I_image = np.swapaxes(I_image, 0, 1)
        I_image = np.swapaxes(I_image, 1, 2)
        I_image[I_image > 1] = 1

        # Now store batches of Images
        scipy.misc.toimage(I_image, cmin=0, cmax=1).save(os.path.join(store_folder, str(batch_counter) + '.png'))

print('Done!')

Evaluating Image 1


/home/dario/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


Done!


In [7]:
import cv2
 
filename = '1.png'
 
image = cv2.imread(filename,0)
print(img)

[[[ 61.  65.  64.]
  [ 81.  85.  84.]
  [ 83.  85.  84.]
  ...
  [171. 149. 135.]
  [171. 149. 135.]
  [171. 149. 135.]]

 [[ 51.  55.  56.]
  [ 74.  78.  79.]
  [ 72.  74.  73.]
  ...
  [173. 151. 137.]
  [174. 152. 138.]
  [174. 152. 138.]]

 [[ 54.  58.  59.]
  [ 82.  86.  87.]
  [ 69.  70.  72.]
  ...
  [179. 158. 139.]
  [179. 158. 139.]
  [179. 158. 139.]]

 ...

 [[ 59.  59.  49.]
  [ 59.  59.  49.]
  [ 59.  59.  51.]
  ...
  [ 72.  75.  58.]
  [ 73.  76.  59.]
  [ 72.  75.  58.]]

 [[ 57.  57.  47.]
  [ 58.  58.  48.]
  [ 57.  57.  49.]
  ...
  [ 71.  74.  57.]
  [ 72.  75.  58.]
  [ 71.  74.  57.]]

 [[ 56.  56.  46.]
  [ 57.  57.  47.]
  [ 57.  57.  49.]
  ...
  [ 71.  74.  57.]
  [ 71.  74.  57.]
  [ 70.  73.  56.]]]
